## Environment Report

**Current Package Versions (Baseline before LLAVA/SHIKRA integration):**

- **Python**: 3.13.3
- **torch**: 2.7.1+cu128
- **torchvision**: 0.22.1+cu128
- **torchaudio**: 2.7.1+cu128
- **easyocr**: 1.7.2
- **opencv-python**: 4.12.0.88
- **pillow**: 11.2.1
- **numpy**: 2.2.6
- **pyttsx3**: 2.99
- **langdetect**: 1.0.9
- **transformers**: 4.52.3
- **gradio**: 5.31.0
- **CUDA Available**: Yes
- **CUDA Version**: 12.8
- **GPU**: NVIDIA GeForce RTX 2060
- **GPU Memory**: 12.88 GB

# Imports and Inits

In [1]:
import os
import glob
import torch
from PIL import Image
import time
import gc
from transformers import AutoProcessor, LlavaForConditionalGeneration
import easyocr
import pyttsx3
from langdetect import detect
import cv2
import numpy as np

## Moving Hugging Face default Download Dir

# # LLAVA OCR Cell - GPU Accelerated Implementation
# import os

# IMPORTANT: Set cache BEFORE any imports from transformers/huggingface
os.environ['HF_HOME'] = 'D:/HuggingFaceCache'
os.environ['TRANSFORMERS_CACHE'] = 'D:/HuggingFaceCache/transformers'
os.environ['HUGGINGFACE_HUB_CACHE'] = 'D:/HuggingFaceCache/hub'

# # Now import everything else
# import glob
# import torch
# from PIL import Image
# import time
# import gc
# from transformers import AutoProcessor, LlavaForConditionalGeneration

# Force GPU usage
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
torch.backends.cudnn.benchmark = True

# Check GPU availability
if not torch.cuda.is_available():
    print("ERROR: CUDA not available. LLAVA requires GPU acceleration.")
    exit(1)

print("=== LLAVA OCR Processing ===")
print(f"GPU: {torch.cuda.get_device_name(0)}")
print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
print(f"HuggingFace cache location: {os.environ.get('HF_HOME', 'default')}")

d:\Repositories\Apziva Projects\Project 4\Mon-Reader\envs\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


=== LLAVA OCR Processing ===
GPU: NVIDIA GeForce RTX 2060
GPU Memory: 12.88 GB
HuggingFace cache location: D:/HuggingFaceCache


# Simple TTS using pyttsx3 Gpu based:

In [ ]:
# Force GPU usage and optimization for RTX 2060
os.environ['CUDA_VISIBLE_DEVICES'] = '0'  # Use first GPU
torch.backends.cudnn.benchmark = True  # Enable cudnn autotuner for performance

# Check GPU and setup
if not torch.cuda.is_available():
    print("ERROR: CUDA not available. This script requires GPU acceleration.")
    print("Please check your NVIDIA drivers and PyTorch installation.")
    exit(1)

# Display GPU information
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA version: {torch.version.cuda}")
print(f"GPU detected: {torch.cuda.get_device_name(0)}")
print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

# Clear GPU memory before starting
torch.cuda.empty_cache()
gc.collect()

# Initialize EasyOCR reader with explicit GPU settings
print("Initializing EasyOCR with GPU acceleration...")
reader = easyocr.Reader(
    ["ar", "ur", "en"], 
    gpu=True,
    verbose=False,
    # For RTX 2060, set reasonable batch size and model parameters
    detector=True,
    recognizer=True
)

# Initialize TTS engine
tts_engine = pyttsx3.init()

def log_gpu_memory():
    """Log current GPU memory usage"""
    allocated = torch.cuda.memory_allocated(0) / 1e9
    reserved = torch.cuda.memory_reserved(0) / 1e9
    print(f"GPU Memory: {allocated:.2f} GB allocated, {reserved:.2f} GB reserved")

def process_images_ocr_save_text(images_folder, output_folder):
    """
    Process all images in the folder with GPU-accelerated OCR and save to text files
    """
    # Create output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        print(f"Created output folder: {output_folder}")
    
    # Get all image files in the folder
    image_extensions = ["*.jpg", "*.jpeg", "*.png", "*.bmp", "*.tiff"]
    image_files = []

    for ext in image_extensions:
        image_files.extend(glob.glob(os.path.join(images_folder, ext)))
        image_files.extend(glob.glob(os.path.join(images_folder, ext.upper())))

    if not image_files:
        print(f"No images found in {images_folder}")
        return

    print(f"Found {len(image_files)} images to process")
    
    # Report initial GPU memory
    log_gpu_memory()

    # Process files
    for i, image_path in enumerate(image_files, 1):
        image_filename = os.path.basename(image_path)
        base_name = os.path.splitext(image_filename)[0]
        text_file_path = os.path.join(output_folder, f"{base_name}.txt")
        
        print(f"\n--- Processing Image {i}/{len(image_files)}: {image_filename} ---")

        try:
            # Read image
            image = cv2.imread(image_path)
            if image is None:
                print(f"Could not read image: {image_path}")
                continue

            # Optimize image for GPU processing (resize large images)
            h, w = image.shape[:2]
            max_dim = 2000  # Optimal for RTX 2060 memory
            if max(h, w) > max_dim:
                scale = max_dim / max(h, w)
                image = cv2.resize(image, (int(w * scale), int(h * scale)))
                print(f"Resized image to {image.shape[1]}x{image.shape[0]} to optimize GPU memory")
            
            # Report GPU memory before OCR
            log_gpu_memory()

            # Extract text using GPU-accelerated EasyOCR
            print("Extracting text with GPU acceleration...")
            start_time = time.time()
            
            # For RTX 2060, use appropriate batch size
            results = reader.readtext(
                image,
                batch_size=2,  # Adjust based on your GPU memory
                paragraph=True,  # Group text into paragraphs
                detail=0  # 0 for more accuracy
            )

            end_time = time.time()
            print(f"OCR completed in {end_time - start_time:.2f} seconds")

            # Combine all detected text with confidence filtering
            extracted_text = " ".join(
                [result[1] for result in results if result[2] > 0.5]
            )  # confidence > 0.5

            if not extracted_text.strip():
                print("No text detected in this image")
                continue

            print(f"Extracted text: {extracted_text}")
            
            # Try to detect language
            try:
                detected_lang = detect(extracted_text)
                print(f"Detected language: {detected_lang}")
                
                # Save text with language information to file
                with open(text_file_path, 'w', encoding='utf-8') as text_file:
                    text_file.write(f"LANG:{detected_lang}\n")
                    text_file.write(extracted_text)
                
                print(f"Saved text to: {text_file_path}")
                
            except Exception as lang_error:
                print(f"Language detection failed: {lang_error}")
                # Save text without language information
                with open(text_file_path, 'w', encoding='utf-8') as text_file:
                    text_file.write(f"LANG:unknown\n")
                    text_file.write(extracted_text)
                
                print(f"Saved text to: {text_file_path} (language unknown)")

            # Clear GPU memory after each image
            torch.cuda.empty_cache()
            
        except Exception as e:
            print(f"Error processing {image_path}: {e}")
            # Clear GPU memory on error
            torch.cuda.empty_cache()
            continue

    # Final GPU memory cleanup
    torch.cuda.empty_cache()
    gc.collect()
    print("\n--- OCR Processing Complete ---")
    log_gpu_memory()
    
    return output_folder

def read_text_files_aloud(text_folder):
    """
    Read all text files in the folder aloud using TTS, one by one with clear separation
    """
    # Get all text files
    text_files = glob.glob(os.path.join(text_folder, "*.txt"))
    
    if not text_files:
        print(f"No text files found in {text_folder}")
        return
    
    print(f"\nFound {len(text_files)} text files to read")
    
    # Language mapping for TTS
    lang_mapping = {
        "en": "english",
        "ar": "arabic",
        "ur": "urdu",
        "hi": "hindi",
        "fa": "persian",
        "ps": "pashto"
    }
    
    # Sort text files alphabetically to ensure consistent reading order
    text_files.sort()
    
    for i, text_file_path in enumerate(text_files, 1):
        file_name = os.path.basename(text_file_path)
        print(f"\n===== Reading File {i}/{len(text_files)}: {file_name} =====")
        
        # Announce the file being read (optional)
        tts_engine.setProperty("rate", 150)
        announcement = f"Reading file {i} of {len(text_files)}: {os.path.splitext(file_name)[0]}"
        print(announcement)
        tts_engine.say(announcement)
        tts_engine.runAndWait()
        
        # Pause between announcement and content
        time.sleep(1)
        
        try:
            # Read text file
            with open(text_file_path, 'r', encoding='utf-8') as file:
                lines = file.readlines()
                
            if not lines:
                print(f"File is empty: {text_file_path}")
                continue
                
            # Extract language information from first line
            lang_line = lines[0].strip()
            if lang_line.startswith("LANG:"):
                detected_lang = lang_line[5:]
                print(f"Language: {detected_lang}")
                # Remove the language line
                content = "".join(lines[1:])
            else:
                # No language information, treat all lines as content
                detected_lang = "unknown"
                content = "".join(lines)
            
            if not content.strip():
                print("No content to read")
                continue
                
            # Print a preview of the content
            content_preview = content[:100] + "..." if len(content) > 100 else content
            print(f"Text content: {content_preview}")
            
            # Get available voices
            voices = tts_engine.getProperty("voices")
            
            # Try to set appropriate voice based on language
            voice_set = False
            for voice in voices:
                tts_lang = lang_mapping.get(detected_lang, "english")
                if (tts_lang.lower() in voice.name.lower() or 
                    detected_lang in voice.id.lower()):
                    tts_engine.setProperty("voice", voice.id)
                    voice_set = True
                    print(f"Using voice: {voice.name}")
                    break
            
            if not voice_set:
                print(f"No specific voice found for {detected_lang}, using default")
            
            # Adjust speech rate based on language
            if detected_lang in ["ar", "ur"]:
                # Slower for Arabic and Urdu
                tts_engine.setProperty("rate", 130)
            else:
                tts_engine.setProperty("rate", 150)
            
            # Read the text aloud
            print(f"Reading text aloud...")
            tts_engine.say(content)
            tts_engine.runAndWait()
            
            # Pause between files to clearly separate them
            print("Finished reading file.")
            time.sleep(2)
            
        except Exception as e:
            print(f"Error reading {text_file_path}: {e}")
            continue
    
    print("\n===== All Text Files Have Been Read =====")

def get_user_confirmation():
    """
    Ask the user if they want to proceed to the TTS reading phase
    """
    while True:
        response = input("\nOCR processing complete. Proceed with reading text files? (y/n): ").lower()
        if response in ['y', 'yes']:
            return True
        elif response in ['n', 'no']:
            return False
        else:
            print("Please enter 'y' or 'n'")

# Main process
if __name__ == "__main__":
    try:
        # Define folders
        images_folder = "part_2_images"
        output_folder = "extracted_text"
        
        # Print GPU optimization message
        print("=== Running GPU-Optimized OCR for RTX 2060 ===")
        
        # Step 1: Process images with OCR and save text
        start_time = time.time()
        text_folder = process_images_ocr_save_text(images_folder, output_folder)
        end_time = time.time()
        
        print(f"OCR processing completed in {end_time - start_time:.2f} seconds")
        
        # Optional: Ask for user confirmation before proceeding to TTS
        if get_user_confirmation():
            # Step 2: Read the saved text files aloud one by one
            read_text_files_aloud(text_folder)
        else:
            print("TTS reading canceled. Text files are saved in the output folder.")
        
        print("Process completed successfully!")
        
    except KeyboardInterrupt:
        print("\nProcess interrupted by user")
        # Clean up GPU memory on interrupt
        torch.cuda.empty_cache()
    except Exception as e:
        print(f"An error occurred: {e}")
        # Clean up GPU memory on error
        torch.cuda.empty_cache()
    finally:
        # Final cleanup
        torch.cuda.empty_cache()
        gc.collect()

In [ ]:
# Try a smaller vision model first
processor = AutoProcessor.from_pretrained("microsoft/git-base", use_fast=True)
print("Downloaded!")

# LLava Model


In [ ]:
# # LLAVA OCR Cell - GPU Accelerated Implementation

# # Force GPU usage
# os.environ['CUDA_VISIBLE_DEVICES'] = '0'
# os.environ['HF_HOME'] = 'D:/HuggingFaceCache' 
# torch.backends.cudnn.benchmark = True

# # Check GPU availability
# if not torch.cuda.is_available():
#     print("ERROR: CUDA not available. LLAVA requires GPU acceleration.")
#     exit(1)

# print("=== LLAVA OCR Processing ===")
# print(f"GPU: {torch.cuda.get_device_name(0)}")
# print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

# def process_images_with_llava(images_folder="part_2_images", output_folder="extracted_text/llava"):
#     """
#     Process images using LLAVA model for OCR with GPU acceleration
#     """
#     # Create output directory if it doesn't exist
#     os.makedirs(output_folder, exist_ok=True)
#     print(f"Output folder: {output_folder}")
    
#     # # Clear GPU memory before starting
#     # torch.cuda.empty_cache()
#     # gc.collect()
    
#     # Initialize LLAVA model and processor
#     print("Loading LLAVA model...")
#     try:
#         # TODO: Replace with your specific LLAVA model path/name
#         model_name = "llava-hf/llava-1.5-7b-hf"  # Example model name
        
#         processor = AutoProcessor.from_pretrained(model_name, use_fast=True)
#         model = LlavaForConditionalGeneration.from_pretrained(
#             model_name,
#             torch_dtype=torch.float16,  # Use fp16 for RTX 2060 efficiency
#             device_map="cuda",
#             low_cpu_mem_usage=True
#         )
#         model.eval()
#         print("LLAVA model loaded successfully!")
        
#     except Exception as e:
#         print(f"Error loading LLAVA model: {e}")
#         print("Please ensure you have the correct model name/path")
#         return
    
#     # Get all image files
#     image_extensions = ["*.jpg", "*.jpeg", "*.png", "*.bmp", "*.tiff"]
#     image_files = []
    
#     for ext in image_extensions:
#         image_files.extend(glob.glob(os.path.join(images_folder, ext)))
#         image_files.extend(glob.glob(os.path.join(images_folder, ext.upper())))
    
#     if not image_files:
#         print(f"No images found in {images_folder}")
#         return
    
#     print(f"Found {len(image_files)} images to process")
    
#     # Process each image
#     processed_count = 0
#     skipped_count = 0
    
#     for i, image_path in enumerate(image_files, 1):
#         image_filename = os.path.basename(image_path)
#         base_name = os.path.splitext(image_filename)[0]
#         text_file_path = os.path.join(output_folder, f"{base_name}.txt")
        
#         # Check if text file already exists (efficiency check)
#         if os.path.exists(text_file_path):
#             print(f"\n[{i}/{len(image_files)}] Skipping {image_filename} - already processed")
#             skipped_count += 1
#             continue
        
#         print(f"\n[{i}/{len(image_files)}] Processing {image_filename}...")
        
#         try:
#             # Load and preprocess image
#             image = Image.open(image_path).convert("RGB")
            
#             # Resize large images to optimize GPU memory
#             max_dim = 1024  # Adjust based on your GPU memory
#             if max(image.size) > max_dim:
#                 image.thumbnail((max_dim, max_dim), Image.Resampling.LANCZOS)
#                 print(f"Resized image to {image.size} for GPU optimization")
            
#             # Prepare prompt for OCR task
#             prompt = "USER: <image>\nExtract all text from this image. Provide only the extracted text without any additional commentary.\nASSISTANT:"
            
#             # Process with LLAVA
#             start_time = time.time()
            
#             # TODO: Adjust this section based on your specific LLAVA implementation
#             inputs = processor(prompt, image, return_tensors="pt").to("cuda")
            
#             # Generate text with GPU
#             with torch.no_grad():
#                 generated_ids = model.generate(
#                     **inputs,
#                     max_new_tokens=1024,
#                     temperature=0.1,  # Low temperature for more accurate OCR
#                     do_sample=False,
#                     use_cache=True
#                 )
            
#             # Decode the generated text
#             extracted_text = processor.decode(generated_ids[0], skip_special_tokens=True)
            
#             # Remove the prompt from the output
#             if "ASSISTANT:" in extracted_text:
#                 extracted_text = extracted_text.split("ASSISTANT:")[-1].strip()
            
#             end_time = time.time()
#             print(f"OCR completed in {end_time - start_time:.2f} seconds")
            
#             if not extracted_text.strip():
#                 print("No text detected in this image")
#                 # Save empty file to avoid reprocessing
#                 with open(text_file_path, 'w', encoding='utf-8') as f:
#                     f.write("")
#                 continue
            
#             # Save extracted text
#             with open(text_file_path, 'w', encoding='utf-8') as f:
#                 f.write(extracted_text)
            
#             print(f"Saved text to: {text_file_path}")
#             print(f"Text preview: {extracted_text[:100]}..." if len(extracted_text) > 100 else f"Text: {extracted_text}")
            
#             processed_count += 1
            
#             # Clear GPU memory after each image
#             torch.cuda.empty_cache()
            
#         except Exception as e:
#             print(f"Error processing {image_filename}: {e}")
#             # Save error file to avoid reprocessing
#             with open(text_file_path, 'w', encoding='utf-8') as f:
#                 f.write(f"ERROR: {str(e)}")
#             torch.cuda.empty_cache()
#             continue
    
#     # Final cleanup
#     del model
#     del processor
#     torch.cuda.empty_cache()
#     gc.collect()
    
#     print(f"\n=== LLAVA Processing Complete ===")
#     print(f"Processed: {processed_count} images")
#     print(f"Skipped: {skipped_count} images (already processed)")
#     print(f"Total: {len(image_files)} images")

# # Run LLAVA OCR processing
# if __name__ == "__main__":
#     process_images_with_llava()



# LLAVA OCR Cell - GPU Accelerated Implementation

import os
import gc
import time
import glob
import torch
from PIL import Image
from transformers import AutoProcessor, LlavaForConditionalGeneration
import warnings
warnings.filterwarnings("ignore")

# Force GPU usage
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['HF_HOME'] = 'D:/HuggingFaceCache' 
torch.backends.cudnn.benchmark = True

# Check GPU availability
device = "cuda" if torch.cuda.is_available() else "cpu"
use_gpu = torch.cuda.is_available()

print("=== LLAVA OCR Processing ===")
if use_gpu:
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    print("Using GPU acceleration")
else:
    print("GPU not available - using CPU (will be slower)")
    print("For better performance, install CUDA-compatible PyTorch")

def process_images_with_llava(images_folder="part_2_images", output_folder="extracted_text/llava"):
    """
    Process images using LLAVA model for OCR with GPU acceleration
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)
    print(f"Output folder: {output_folder}")
    
    # Clear GPU memory before starting (only if GPU available)
    if use_gpu:
        torch.cuda.empty_cache()
        gc.collect()
    
    # Initialize LLAVA model and processor
    print("Loading LLAVA model...")
    try:
        # Updated model name - use the correct LLAVA model
        model_name = "llava-hf/llava-1.5-7b-hf"
        
        print("Downloading/Loading processor...")
        processor = AutoProcessor.from_pretrained(
            model_name, 
            use_fast=True,
            trust_remote_code=True
        )
        
        print("Downloading/Loading model...")
        model = LlavaForConditionalGeneration.from_pretrained(
            model_name,
            torch_dtype=torch.float16 if use_gpu else torch.float32,  # Use fp16 only if GPU available
            device_map="auto" if use_gpu else None,  # Use auto device mapping only if GPU available
            low_cpu_mem_usage=True,
            trust_remote_code=True
        )
        
        # Move model to appropriate device
        if use_gpu and not next(model.parameters()).is_cuda:
            model = model.to(device)
        elif not use_gpu:
            model = model.to(device)
        
        model.eval()
        print("LLAVA model loaded successfully!")
        
    except Exception as e:
        print(f"Error loading LLAVA model: {e}")
        print("Trying alternative approach...")
        
        # Alternative: Try different model or installation
        try:
            # Alternative model names to try
            alternative_models = [
                "llava-hf/llava-1.5-13b-hf",
                "llava-hf/llava-v1.6-mistral-7b-hf",
                "llava-hf/llava-v1.6-vicuna-7b-hf"
            ]
            
            for alt_model in alternative_models:
                try:
                    print(f"Trying {alt_model}...")
                    processor = AutoProcessor.from_pretrained(alt_model, trust_remote_code=True)
                    model = LlavaForConditionalGeneration.from_pretrained(
                        alt_model,
                        torch_dtype=torch.float16 if use_gpu else torch.float32,
                        device_map="auto" if use_gpu else None,
                        low_cpu_mem_usage=True,
                        trust_remote_code=True
                    )
                    if use_gpu and not next(model.parameters()).is_cuda:
                        model = model.to(device)
                    elif not use_gpu:
                        model = model.to(device)
                    model.eval()
                    print(f"Successfully loaded {alt_model}!")
                    model_name = alt_model
                    break
                except Exception as alt_e:
                    print(f"Failed to load {alt_model}: {alt_e}")
                    continue
            else:
                print("All model loading attempts failed.")
                print("Please ensure you have installed the requirements:")
                print("pip install transformers torch torchvision accelerate")
                print("pip install git+https://github.com/huggingface/transformers.git")
                return
                
        except Exception as final_e:
            print(f"Final error: {final_e}")
            return
    
    # Get all image files
    image_extensions = ["*.jpg", "*.jpeg", "*.png", "*.bmp", "*.tiff"]
    image_files = []
    
    for ext in image_extensions:
        image_files.extend(glob.glob(os.path.join(images_folder, ext)))
        image_files.extend(glob.glob(os.path.join(images_folder, ext.upper())))
    
    if not image_files:
        print(f"No images found in {images_folder}")
        return
    
    print(f"Found {len(image_files)} images to process")
    
    # Process each image
    processed_count = 0
    skipped_count = 0
    
    for i, image_path in enumerate(image_files, 1):
        image_filename = os.path.basename(image_path)
        base_name = os.path.splitext(image_filename)[0]
        text_file_path = os.path.join(output_folder, f"{base_name}.txt")
        
        # Check if text file already exists (efficiency check)
        if os.path.exists(text_file_path):
            print(f"\n[{i}/{len(image_files)}] Skipping {image_filename} - already processed")
            skipped_count += 1
            continue
        
        print(f"\n[{i}/{len(image_files)}] Processing {image_filename}...")
        
        try:
            # Load and preprocess image
            image = Image.open(image_path).convert("RGB")
            
            # Resize large images to optimize memory
            max_dim = 1024 if use_gpu else 512  # Smaller images for CPU processing
            if max(image.size) > max_dim:
                image.thumbnail((max_dim, max_dim), Image.Resampling.LANCZOS)
                print(f"Resized image to {image.size} for {'GPU' if use_gpu else 'CPU'} optimization")
            
            # Prepare prompt for OCR task - Updated format
            prompt = "USER: <image>\nExtract all text from this image. Provide only the extracted text without any additional commentary.\nASSISTANT:"
            
            # Process with LLAVA
            start_time = time.time()
            
            # Updated processing approach
            inputs = processor(prompt, image, return_tensors="pt")
            
            # Move inputs to appropriate device
            if use_gpu:
                inputs = {k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in inputs.items()}
            else:
                inputs = {k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in inputs.items()}
            
            # Generate text with appropriate device
            with torch.no_grad():
                generated_ids = model.generate(
                    **inputs,
                    max_new_tokens=1024 if use_gpu else 512,  # Reduce tokens for CPU
                    temperature=0.1,  # Low temperature for more accurate OCR
                    do_sample=False,
                    use_cache=True,
                    pad_token_id=processor.tokenizer.eos_token_id
                )
            
            # Decode the generated text
            generated_text = processor.decode(generated_ids[0], skip_special_tokens=True)
            
            # Clean up the output - remove prompt
            if "ASSISTANT:" in generated_text:
                extracted_text = generated_text.split("ASSISTANT:")[-1].strip()
            else:
                extracted_text = generated_text.strip()
            
            # Remove any remaining prompt artifacts
            if "USER:" in extracted_text:
                extracted_text = extracted_text.split("USER:")[-1].strip()
            
            end_time = time.time()
            print(f"OCR completed in {end_time - start_time:.2f} seconds")
            
            if not extracted_text.strip():
                print("No text detected in this image")
                # Save empty file to avoid reprocessing
                with open(text_file_path, 'w', encoding='utf-8') as f:
                    f.write("")
                continue
            
            # Save extracted text
            with open(text_file_path, 'w', encoding='utf-8') as f:
                f.write(extracted_text)
            
            print(f"Saved text to: {text_file_path}")
            print(f"Text preview: {extracted_text[:100]}..." if len(extracted_text) > 100 else f"Text: {extracted_text}")
            
            processed_count += 1
            
            # Clear memory after each image (only if GPU available)
            if use_gpu:
                torch.cuda.empty_cache()
            
        except Exception as e:
            print(f"Error processing {image_filename}: {e}")
            # Save error file to avoid reprocessing
            with open(text_file_path, 'w', encoding='utf-8') as f:
                f.write(f"ERROR: {str(e)}")
            if use_gpu:
                torch.cuda.empty_cache()
            continue
    
    # Final cleanup
    del model
    del processor
    if use_gpu:
        torch.cuda.empty_cache()
    gc.collect()
    
    print(f"\n=== LLAVA Processing Complete ===")
    print(f"Device used: {'GPU' if use_gpu else 'CPU'}")
    print(f"Processed: {processed_count} images")
    print(f"Skipped: {skipped_count} images (already processed)")
    print(f"Total: {len(image_files)} images")

# Install requirements function
def install_requirements():
    """Install required packages if not already installed"""
    import subprocess
    import sys
    
    required_packages = [
        "transformers>=4.36.0",
        "torch",
        "torchvision", 
        "accelerate",
        "pillow",
        "bitsandbytes"  # For efficient loading
    ]
    
    print("Installing/updating required packages...")
    for package in required_packages:
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        except subprocess.CalledProcessError as e:
            print(f"Failed to install {package}: {e}")
    
    # Install latest transformers from git for LLAVA support
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "git+https://github.com/huggingface/transformers.git"])
    except subprocess.CalledProcessError as e:
        print(f"Failed to install transformers from git: {e}")

# Run LLAVA OCR processing
if __name__ == "__main__":
    # Uncomment the line below if you need to install requirements
    # install_requirements()
    
    process_images_with_llava()

d:\Repositories\Apziva Projects\Project 4\Mon-Reader\envs\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


=== LLAVA OCR Processing ===
GPU: NVIDIA GeForce RTX 2060
GPU Memory: 12.88 GB
Using GPU acceleration
Output folder: extracted_text/llava
Loading LLAVA model...
Downloading/Loading processor...
Downloading/Loading model...


Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

# Shikra Model

In [2]:
# SHIKRA OCR Cell - GPU Accelerated Implementation
import os
import glob
import torch
from PIL import Image
import time
import gc
from transformers import AutoProcessor, AutoModelForVision2Seq

# Force GPU usage
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
torch.backends.cudnn.benchmark = True

# Check GPU availability
if not torch.cuda.is_available():
    print("ERROR: CUDA not available. SHIKRA requires GPU acceleration.")
    exit(1)

print("=== SHIKRA OCR Processing ===")
print(f"GPU: {torch.cuda.get_device_name(0)}")
print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

def process_images_with_shikra(images_folder="part_2_images", output_folder="extracted_text/shikra"):
    """
    Process images using SHIKRA model for OCR with GPU acceleration
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)
    print(f"Output folder: {output_folder}")
    
    # Clear GPU memory before starting
    torch.cuda.empty_cache()
    gc.collect()
    
    # Initialize SHIKRA model and processor
    print("Loading SHIKRA model...")
    try:
        # TODO: Replace with your specific SHIKRA model path/name
        model_name = "shikras/shikra-7b-v1"  # Example model name
        
        processor = AutoProcessor.from_pretrained(model_name, use_fast=True)
        model = AutoModelForVision2Seq.from_pretrained(
            model_name,
            torch_dtype=torch.float16,  # Use fp16 for RTX 2060 efficiency
            device_map="cuda",
            low_cpu_mem_usage=True
        )
        model.eval()
        print("SHIKRA model loaded successfully!")
        
    except Exception as e:
        print(f"Error loading SHIKRA model: {e}")
        print("Please ensure you have the correct model name/path")
        return
    
    # Get all image files
    image_extensions = ["*.jpg", "*.jpeg", "*.png", "*.bmp", "*.tiff"]
    image_files = []
    
    for ext in image_extensions:
        image_files.extend(glob.glob(os.path.join(images_folder, ext)))
        image_files.extend(glob.glob(os.path.join(images_folder, ext.upper())))
    
    if not image_files:
        print(f"No images found in {images_folder}")
        return
    
    print(f"Found {len(image_files)} images to process")
    
    # Process each image
    processed_count = 0
    skipped_count = 0
    
    for i, image_path in enumerate(image_files, 1):
        image_filename = os.path.basename(image_path)
        base_name = os.path.splitext(image_filename)[0]
        text_file_path = os.path.join(output_folder, f"{base_name}.txt")
        
        # Check if text file already exists (efficiency check)
        if os.path.exists(text_file_path):
            print(f"\n[{i}/{len(image_files)}] Skipping {image_filename} - already processed")
            skipped_count += 1
            continue
        
        print(f"\n[{i}/{len(image_files)}] Processing {image_filename}...")
        
        try:
            # Load and preprocess image
            image = Image.open(image_path).convert("RGB")
            
            # Resize large images to optimize GPU memory
            max_dim = 1024  # Adjust based on your GPU memory
            if max(image.size) > max_dim:
                image.thumbnail((max_dim, max_dim), Image.Resampling.LANCZOS)
                print(f"Resized image to {image.size} for GPU optimization")
            
            # Prepare prompt for OCR task
            # SHIKRA might use a different prompt format - adjust as needed
            prompt = "<image> Extract and transcribe all text visible in this image."
            
            # Process with SHIKRA
            start_time = time.time()
            
            # TODO: Adjust this section based on your specific SHIKRA implementation
            inputs = processor(
                text=prompt,
                images=image,
                return_tensors="pt"
            ).to("cuda")
            
            # Generate text with GPU
            with torch.no_grad():
                generated_ids = model.generate(
                    **inputs,
                    max_new_tokens=1024,
                    temperature=0.1,  # Low temperature for more accurate OCR
                    do_sample=False,
                    num_beams=1,  # Adjust based on accuracy vs speed tradeoff
                    use_cache=True
                )
            
            # Decode the generated text
            extracted_text = processor.decode(generated_ids[0], skip_special_tokens=True)
            
            # Clean up the output (remove prompt if included)
            if prompt in extracted_text:
                extracted_text = extracted_text.replace(prompt, "").strip()
            
            end_time = time.time()
            print(f"OCR completed in {end_time - start_time:.2f} seconds")
            
            if not extracted_text.strip():
                print("No text detected in this image")
                # Save empty file to avoid reprocessing
                with open(text_file_path, 'w', encoding='utf-8') as f:
                    f.write("")
                continue
            
            # Save extracted text
            with open(text_file_path, 'w', encoding='utf-8') as f:
                f.write(extracted_text)
            
            print(f"Saved text to: {text_file_path}")
            print(f"Text preview: {extracted_text[:100]}..." if len(extracted_text) > 100 else f"Text: {extracted_text}")
            
            processed_count += 1
            
            # Clear GPU memory after each image
            torch.cuda.empty_cache()
            
        except Exception as e:
            print(f"Error processing {image_filename}: {e}")
            # Save error file to avoid reprocessing
            with open(text_file_path, 'w', encoding='utf-8') as f:
                f.write(f"ERROR: {str(e)}")
            torch.cuda.empty_cache()
            continue
    
    # Final cleanup
    del model
    del processor
    torch.cuda.empty_cache()
    gc.collect()
    
    print(f"\n=== SHIKRA Processing Complete ===")
    print(f"Processed: {processed_count} images")
    print(f"Skipped: {skipped_count} images (already processed)")
    print(f"Total: {len(image_files)} images")

# Run SHIKRA OCR processing
if __name__ == "__main__":
    process_images_with_shikra()

=== SHIKRA OCR Processing ===
GPU: NVIDIA GeForce RTX 2060
GPU Memory: 12.88 GB
Output folder: extracted_text/shikra
Loading SHIKRA model...
Error loading SHIKRA model: shikras/shikra-7b-v1 is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`
Please ensure you have the correct model name/path
